In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        

/kaggle/input/pre-train-data/pre_train.csv
/kaggle/input/llm-detect-ai-generated-text/sample_submission.csv
/kaggle/input/llm-detect-ai-generated-text/train_prompts.csv
/kaggle/input/llm-detect-ai-generated-text/test_essays.csv
/kaggle/input/llm-detect-ai-generated-text/train_essays.csv
/kaggle/input/daigt-one-place-all-data/concatenated.csv


In [2]:
df1= pd.read_csv("/kaggle/input/daigt-one-place-all-data/concatenated.csv")
train_p = pd.read_csv("/kaggle/input/llm-detect-ai-generated-text/train_prompts.csv")
train_e=pd.read_csv("/kaggle/input/llm-detect-ai-generated-text/train_essays.csv")
test = pd.read_csv("/kaggle/input/llm-detect-ai-generated-text/test_essays.csv")
sample = pd.read_csv("/kaggle/input/llm-detect-ai-generated-text/sample_submission.csv")

In [3]:
df1.drop(columns=['id','prompt_id','model','kaggle_repo'],inplace = True)

In [4]:
merged_df = pd.merge(train_e, train_p, on='prompt_id')
merged_df['text'] = merged_df['text']+' '+merged_df['source_text'] + ' ' + merged_df['instructions']
df = merged_df.drop(columns = ['id','prompt_name','instructions','source_text'])
df.drop(columns = ['prompt_id'],inplace = True)

In [5]:
train = pd.concat([df , df1], axis =0)

In [6]:
train

,text,generated
0,Cars. Cars have been around since they became ...,0
1,Transportation is a large necessity in most co...,0
2,"""America's love affair with it's vehicles seem...",0
3,How often do you ride in a car? Do you drive a...,0
4,Cars are a wonderful thing. They are perhaps o...,0
...,...,...
54686,I believe using cellphones in class for educat...,0
54687,"Working alone, students do not have to argue w...",0
54688,"""A problem is a chance for you to do your best...",0
54689,Many people disagree with Albert Schweitzer's ...,0


In [7]:
import re
import string
import nltk
import spacy
eng = spacy.load("en_core_web_lg")
from nltk.stem import PorterStemmer
porter = PorterStemmer()

In [8]:
def preprocess(text):
    doc = eng(text)
    no_stop_words = [token.text.lower() for token in doc if not token.is_stop and not token.is_punct and not token.like_url and not token.is_digit ]
    stemmed_tokens = [porter.stem(token) for token in no_stop_words]
    return " ".join(no_stop_words)

In [9]:
from tqdm import tqdm
tqdm.pandas()

In [10]:
# train['final_txt'] = train['text'].progress_apply(preprocess)
test['text'] = test['text'].progress_apply(preprocess)

100%|██████████| 3/3 [00:00<00:00, 77.92it/s]


In [11]:
# train.to_csv("pre_train.csv")

In [12]:
train = pd.read_csv("/kaggle/input/pre-train-data/pre_train.csv")

In [13]:
train.drop(columns=['Unnamed: 0'],inplace = True)

In [14]:
train

,text,generated
0,car car around sinc becam famou 1900 henri for...,0
1,transport larg necess countri worldwid doubt c...,0
2,america love affair vehicl seem cool say elisa...,0
3,often ride car drive one motor vehicl work sto...,0
4,car wonder thing perhap one world greatest adv...,0
...,...,...
56064,believ use cellphon class educ us goodbad idea...,0
56065,work alon student argu deciss proyect work som...,0
56066,problem chanc best think quot cant best everi ...,0
56067,mani peopl disagre albert schweitzer quot exam...,0


In [15]:
y = train['generated']

In [16]:
from gensim.models import Word2Vec
vector = Word2Vec(window=10,min_count=2)

In [18]:
nlp = [nltk.word_tokenize(i) for i in train['text']]
comp = nlp 

In [19]:
vector.build_vocab(comp)

In [20]:
vector.train(comp, total_examples=vector.corpus_count, epochs=10)

(118738623, 130883810)

In [21]:
model  = vector.wv

In [22]:
def get_sentence_vector(word_list):
    word_vectors = [vector.wv[word] for word in word_list if word in vector.wv]
    if len(word_vectors) > 0:
        return np.mean(word_vectors, axis=0)
    else:
        return np.zeros(word2vec_model.vector_size)
    
#The below code works very slow, so I have made my own function to get mean vector.
    
# def vectors(text):
#     doc = eng(text)
#     filtered_tokens = []
#     for token in doc:
#         filtered_tokens.append(token.text)
#     return model.get_mean_vector(filtered_tokens)

# train ['vector'] = train['final_txt'].progress_apply(lambda x: vectors(x))
# test['vector'] = test['final_txt'].progress_apply(lambda x: vectors(x))

In [23]:
train['final_txt'] = train['text'].progress_apply(get_sentence_vector).tolist()
test['final_txt'] = test['text'].progress_apply(get_sentence_vector).tolist()

100%|██████████| 3/3 [00:00<00:00, 3040.82it/s]


In [24]:
train

,text,generated,final_txt
0,car car around sinc becam famou 1900 henri for...,0,"[1.8256629, -0.24490729, 2.32728, 0.681299, -0..."
1,transport larg necess countri worldwid doubt c...,0,"[1.8164363, -0.21497463, 2.3080523, 0.71222544..."
2,america love affair vehicl seem cool say elisa...,0,"[1.8126339, -0.21533021, 2.2869043, 0.7138106,..."
3,often ride car drive one motor vehicl work sto...,0,"[1.820542, -0.2438672, 2.3464384, 0.6903707, -..."
4,car wonder thing perhap one world greatest adv...,0,"[1.8560257, -0.2598191, 2.396209, 0.73139983, ..."
...,...,...,...
56064,believ use cellphon class educ us goodbad idea...,0,"[1.6472068, 0.19534564, 2.1956253, 1.0482316, ..."
56065,work alon student argu deciss proyect work som...,0,"[1.826534, -0.23131563, 2.5663855, 0.85469335,..."
56066,problem chanc best think quot cant best everi ...,0,"[1.7698623, -0.36180046, 2.5855057, 0.55686224..."
56067,mani peopl disagre albert schweitzer quot exam...,0,"[1.6871347, -0.31109923, 2.396795, 0.5524966, ..."


In [25]:
X = train.copy()

In [26]:
train_2d = np.stack(X['final_txt'])
test_2d =  np.stack(test['final_txt'])

In [27]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(train_2d ,y,test_size=0.2,random_state=2)

VOTING AND BASIC MODELS

In [28]:
# from sklearn.naive_bayes import GaussianNB,MultinomialNB,BernoulliNB
# from sklearn.metrics import accuracy_score,confusion_matrix,precision_score,f1_score,recall_score
# from sklearn.metrics import roc_auc_score
# from sklearn.linear_model import LogisticRegression
# from sklearn.svm import SVC
# from sklearn.naive_bayes import MultinomialNB
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.ensemble import AdaBoostClassifier
# from sklearn.ensemble import BaggingClassifier
# from sklearn.ensemble import ExtraTreesClassifier
# from sklearn.ensemble import GradientBoostingClassifier
# from xgboost import XGBClassifier

# svc = SVC(kernel='sigmoid', gamma=1.0)
# knc = KNeighborsClassifier()
# mnb = MultinomialNB()
# bnb = BernoulliNB()
# dtc = DecisionTreeClassifier(max_depth=5)
# lrc = LogisticRegression(solver='liblinear', penalty='l1')
# rfc = RandomForestClassifier(n_estimators=50, random_state=2)
# abc = AdaBoostClassifier(n_estimators=50, random_state=2)
# bc = BaggingClassifier(n_estimators=50, random_state=2)
# etc = ExtraTreesClassifier(n_estimators=50, random_state=2)
# gbdt = GradientBoostingClassifier(n_estimators=50,random_state=2)
# xgb = XGBClassifier(n_estimators=50,random_state=2)

# from sklearn.model_selection import cross_val_score
# def train_classifier(clf,X_train,y_train,X_test,y_test,X , y):
#     csr_val = cross_val_score(clf, train_2d, y, cv=10 ,scoring= "roc_auc")
#     clf.fit(X_train,y_train)
#     y_pred = clf.predict(X_test)
#     recall = recall_score(y_test,y_pred)
#     precision = precision_score(y_test,y_pred)
#     f1score = f1_score(y_test,y_pred)
#     accuracy = roc_auc_score(y_test,y_pred)
#     return recall,precision ,f1score,accuracy, csr_val.mean()

# clfs = {
# #     'SVC' : svc,
# #     'KN' : knc, 
# #     'NB': mnb, 
# #     'BNB': bnb,
# #     "vote": ensemble,
# #     'DT': dtc, 
# #     'LR': lrc, 
# #     'RF': rfc, 
# #     'AdaBoost': abc, 
# #     'BgC': bc, 
# #     'ETC': etc,
# #     'GBDT':gbdt,
# #     'xgb':xgb
#  }

# for name,clf in clfs.items():
    
#     recall,precision,f1score,accuracy,csr_val = train_classifier(clf, X_train,y_train,X_test,y_test,train_2d , y)
    
#     print("For ",name)
#     print("recall - ",recall)
#     print("Precision - ",precision)
#     print("f1_score - ",f1score)
#     print("roc_auc- ",accuracy)
#     print("cross_val - ",csr_val)
#     print("************************")
    
# # For  vote
# # recall -  0.8367964494654024
# # Precision -  0.8191153238546603
# # f1_score -  0.8278614908691748
# # roc_auc-  0.8452002065131071
# # cross_val -  0.8269816399606456

In [29]:
# # clf = MultinomialNB(alpha=0.02)
# #     clf2 = MultinomialNB(alpha=0.01)
# from sklearn.linear_model import SGDClassifier
# from sklearn.ensemble import VotingClassifier
# from lightgbm import LGBMClassifier
# from catboost import CatBoostClassifier
# sgd_model = SGDClassifier(max_iter=8000, tol=1e-4, loss="modified_huber") 
# p6={'n_iter': 2500,
# 'verbose': -1,
# 'objective': 'cross_entropy',
# 'metric': 'auc',
# 'learning_rate': 0.00581909898961407, 
# 'colsample_bytree': 0.78,
# 'colsample_bynode': 0.8, 
# 'lambda_l1': 4.562963348932286, 
# 'lambda_l2': 2.97485, 
# 'min_data_in_leaf': 115, 
# 'max_depth': 23, 
# 'max_bin': 898}

# lgb=LGBMClassifier(**p6)
# cat=CatBoostClassifier(iterations=2000,
#                    verbose=0,
#                    l2_leaf_reg=6.6591278779517808,
#                    learning_rate=0.005599066836106983,
#                    subsample = 0.4,
#                    allow_const_label=True,
#                    loss_function = 'CrossEntropy')

# weights = [0.31,0.31,0.312]

# ensemble = VotingClassifier(estimators=[
#                                     ('sgd', sgd_model),
#                                     ('lgb',lgb), 
#                                     ('cat', cat)
#                                    ],
#                         weights=weights, voting='soft', n_jobs=-1)
# # ensemble.fit(X_train, y_train)

In [30]:
from catboost import CatBoostClassifier
# from sklearn.model_selection import GridSearchCV

# # Define the CatBoostClassifier
# catboost = CatBoostClassifier(silent = True)

# # Define parameter grid for CatBoostClassifier
# param_grid_catboost = {
#     'iterations': [50, 100, 200],
#     'learning_rate': [0.01, 0.1],
#     'depth': [4, 6, 8],
#     'l2_leaf_reg': [3, 5, 7],
# }

# Perform GridSearchCV for CatBoostClassifier with scoring as ROC AUC
# grid_search_catboost = GridSearchCV(catboost, param_grid_catboost, cv=3, scoring='roc_auc', n_jobs=-1, verbose = 1)
# grid_search_catboost.fit(X_train, y_train)  # Replace X_train and y_train with your training data

# best_params_catboost = grid_search_catboost.best_params_
# best_score_catboost = grid_search_catboost.best_score_

best_params_catboost = {'depth': 8, 'iterations': 200, 'l2_leaf_reg': 5, 'learning_rate': 0.1}
# print("Best Parameters for CatBoostClassifier:", best_params_catboost)
# print("Best ROC AUC Score for CatBoostClassifier:", best_score_catboost)

In [31]:
frequencies = y.value_counts()
weights = {0: frequencies.sum() / frequencies[0], 1: frequencies.sum() / frequencies[1]}
weights

{0: 1.7923726104469024, 1: 2.2620325170452253}

In [32]:
model = CatBoostClassifier(**best_params_catboost, silent = True)
model.fit(train_2d,y)

In [33]:
y_pred = model.predict_proba(test_2d)

In [34]:
submit = pd.DataFrame(sample['id'])
submit['generated'] = y_pred[:,1]
submit

,id,generated
0,0000aaaa,0.702417
1,1111bbbb,0.953273
2,2222cccc,0.821520


In [35]:
submit.to_csv('submission.csv', index=False)